This notebook converts **Starburst99** `.ifaspec` output files into FITS format compatible with **GalspecFitX**. The **Starburst99** software for generating the templates is available [here](https://massivestars.stsci.edu/starburst99/docs/run.html#SPECS).

For guidance on converting **BPASS** templates, see the `bpass_conversion` notebook.

Instructions for incorporating new **Starburst99** and **BPASS** templates into the existing **GalspecFitX** libraries can be found in *Rivera et al. (2025)*.


## Imports

In [1]:
import os
import numpy as np
from astropy.io import fits

## Functions

In [2]:
def fig_to_lam(filename, output_path, evol_track):
    """
    Processes a tab-delimited data file generated by Starburst99 v7.0.1. containing spectral evolution data. 
    This function creates a unique list of wavelengths and ages from the first two columns.
    The wavelength array is saved to a FITS file for the GalSpecFitX library.
    
    Parameters:
    ----------
    filename : str
        Path to the input text file containing the data.
    output_path : str
        Directory where the output FITS file will be saved.
    evol_track : str
        Name of the evolutionary track for which the wavelength array will be generated.

    Returns:
    -------
    ages : list of float
        List of stellar ages (in Gyr) corresponding to time change points in the data.
    
    Notes:
    -----
    - Assumes the input file has header or metadata in the first 6 lines, which are skipped.
    - Assumes the data columns are tab-separated and numerical starting from the 7th line.
    - The FITS file will contain the second column of the data (up to the first 4200 rows).
    - Ages are extracted when a change is detected in the first column (e.g., time).
    """
    
    with open(filename) as file:
        lines = [line.split('\t')[0] for line in file]
        
    data = np.loadtxt(lines[6:])
    
    new_hdul = fits.HDUList()
    new_hdul.append(fits.PrimaryHDU(data[:4200, 1]))
    
    starburst99_lam_file = f"{evol_track}_lam.fits"
    new_hdul.writeto(os.path.join(output_path, evol_track, starburst99_lam_file),overwrite=True)
    
    ages = []
    
    for i in np.arange(len(data) - 1):
        if data[i+1, 0] != data[i,0]:
            ages.append(data[i,0] / 1e9)
            ages.append(data[i+1,0] / 1e9)
            
    
    ages = list(set(ages))
    ages.sort()
            
    return ages, starburst99_lam_file

In [3]:
def fig_to_temp(lam, filename, output_path, evol_track, star_form, star_evol, IMF_name, IMF_slope, Z, ages):
    """
    Processes a tab-delimited output file from Starburst99 v7.0.1 containing spectral evolution data.
    Extracts the unnormalized flux values from the third column and converts them into separate spectral 
    arrays corresponding to different stellar population ages.

    Parameters:
    ----------
    filename : str
        Path to the input file containing spectral data with metadata in the first 6 lines.
    output_path : str
        Output directory where the FITS files will be saved.
    evol_track : str
        Evolutionary track identifier (used in output file naming).
    lam : array_like
        Wavelength grid (used to segment the data by age).
    IMF : str
        Initial Mass Function identifier (used in output file naming).
    Z : str
        Metallicity value as a string (used in output file naming).
    ages : list of str
        List of age strings corresponding to different time steps in the model (used in output file naming).
    star_form : str
        Star formation history identifier (used in output file naming).

    Notes:
    -----
    - Assumes the input file has 6 header lines and then columns of tab-separated data.
    - Uses the third column (index 2) of the data for flux values (in log scale).
    - Output FITS filenames are constructed with the format:
      `{evol_track}_{IMF}.Zp{Z}T{age}_{star_form}.fits`
    - Overwrites existing files with the same names.
    """    
    
    with open(filename) as file:
        lines = [line.split('\t')[0] for line in file]
        
    data = np.loadtxt(lines[6:])
    
    for i in np.arange(0, len(ages)):
        new_hdul = fits.HDUList()
        new_hdul.append(fits.PrimaryHDU(10**data[len(lam)*i:len(lam)*(i+1), 2]))
        new_hdul.writeto(os.path.join(output_path, evol_track, star_form, star_evol, IMF_name, f"{evol_track}_{star_form[:4]}_{star_evol[:3]}_{IMF_slope}.Zp{Z}T{ages[i]}.fits"), overwrite=True)

### Set parameters

The directory structure of the GalspecFitX library is broken down in Rivera et. a (2025). Please keep in mind that the following parameter strings should correspond to the name of an existing directory or will be used to identify the .dat files or for filenaming. 

In [4]:
# Main directory to contain Starburst99 templates
output_path = '/grp/hst/wfc3i/irivera/tsrc/GalSpecFitX/full_suite/STARBURST99/'

In [5]:
# Sub-directory names
evol_track = 'geneva_std' # evolutionary track (see https://massivestars.stsci.edu/starburst99/docs/run.html#IZ)
star_form = 'instantaneous' # star formation (e.g. 'instantaneous' or 'continuous')
star_evol = 'single' # star evolution (e.g. single or binary)
IMF_name = 'kroupa' # IMF name (e.g. salpeter or kroupa)
IMF_slope = '1.30_2.30' # Slope corresponding to IMF name (e.g. 2.35 for 'salpeter' or 1.30_2.30 for 'kroupa')

In [6]:
# Metallicities - must have 3 decimal places for utility script to properly read template
metallicities = ["0.001", "0.004", "0.008", "0.020", "0.040"]

## Create wavelength array file, and format the ages

The ages and wavelengths are extracted from the .ifaspec1 file. The wavelengths are converted to an array and saved as a FITS file. The wavelengths should be in angstroms and the ages in Gyr.

In [7]:
# Path to .ifaspec1 files for one evolutionary track, star formation, star evolution, and IMF with the only difference between the .ifaspec1 files being the metallicities.
ifaspec_file_path = f'../../STARBURST99/{evol_track}_data/{IMF_name}/'
one_ifaspec_file = f'{ifaspec_file_path}/{evol_track}_{IMF_name}_{metallicities[0]}_{star_form}.ifaspec1'

In [8]:
ages, starburst99_lam_file = fig_to_lam(one_ifaspec_file, output_path, evol_track)

In [9]:
ages

[1e-05,
 0.00101,
 0.00201,
 0.00301,
 0.00401,
 0.00501,
 0.00601,
 0.00701,
 0.00801,
 0.00901,
 0.01001,
 0.01101,
 0.01201,
 0.01301,
 0.01401,
 0.01501,
 0.01601,
 0.01701,
 0.01801,
 0.01901,
 0.02001,
 0.02101,
 0.02201,
 0.02301,
 0.02401,
 0.02501,
 0.02601,
 0.02701,
 0.02801,
 0.02901,
 0.03001,
 0.03101,
 0.03201,
 0.03301,
 0.03401,
 0.03501,
 0.03601,
 0.03701,
 0.03801,
 0.03901,
 0.04001,
 0.04101,
 0.04201,
 0.04301,
 0.04401,
 0.04501,
 0.04601,
 0.04701,
 0.04801,
 0.04901,
 0.05001,
 0.05101,
 0.05201,
 0.05301,
 0.05401,
 0.05501,
 0.05601,
 0.05701,
 0.05801,
 0.05901,
 0.06001,
 0.06101,
 0.06201,
 0.06301,
 0.06401,
 0.06501,
 0.06601,
 0.06701,
 0.06801,
 0.06901,
 0.07001,
 0.07101,
 0.07201,
 0.07301,
 0.07401,
 0.07501,
 0.07601,
 0.07701,
 0.07801,
 0.07901,
 0.08001,
 0.08101,
 0.08201,
 0.08301,
 0.08401,
 0.08501,
 0.08601,
 0.08701,
 0.08801,
 0.08901,
 0.09001,
 0.09101,
 0.09201,
 0.09301,
 0.09401,
 0.09501,
 0.09601,
 0.09701,
 0.09801,
 0.09901,
 0

## Create the wavelength array and check the range for consistency

Use one .ifaspec1 file to create the wavelength file for the specified evolutionary track. It shouldn't matter which since all the templates should have the same wavelength sampling.

In [10]:
starburst99_lam = fits.getdata(f"{output_path}/{evol_track}/{starburst99_lam_file}")

In [11]:
# Comparing this to what you expect was extracted from the .ifaspec1 file is a good way to confirm correctness
starburst99_lam

array([ 900.  ,  900.44,  900.88, ..., 2996.89, 2997.67, 2998.45],
      dtype='>f8')

In [12]:
starburst99_lam[0], starburst99_lam[-1]

(900.0, 2998.45)

In [13]:
len(starburst99_lam)

4200

## Create spectral templates from Starburst99 data files for use in GalSpecFitX

The line below converts the unnormalized fluxes column in the .ifaspec1 files into FITS files for use in the code. This expects one evolutionary track, star formation, star evolution, and IMF with the only difference between the .ifaspec1 files being the metallicities.

In [14]:
[fig_to_temp(starburst99_lam, f"{ifaspec_file_path}/{evol_track}_{IMF_name}_{m}_{star_form}.ifaspec1", output_path, evol_track, star_form, star_evol, IMF_name, IMF_slope, f"{m}", ages) for m in metallicities]

[None, None, None, None, None]

In [15]:
# Again another way to verify correctness is compare one template with its original .ifaspec1 file
fits.getdata(f'{output_path}/{evol_track}/{star_form}/{star_evol}/{IMF_name}/{evol_track}_{star_form[:4]}_{star_evol[:3]}_{IMF_slope}.Zp{metallicities[0]}T{ages[0]}.fits')

array([1.23355921e+40, 1.23228170e+40, 1.23058042e+40, ...,
       1.26368840e+39, 1.26339746e+39, 1.26310659e+39], dtype='>f8')